In [3]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import tiktoken
import os
import keyring

In [5]:
# Step 1. Load the document
file_path = os.path.join(os.getcwd(), 'docs', 'faiss.pdf')
raw_documents = PyPDFLoader(file_path=file_path).load()

In [7]:
# Step 2. Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

In [9]:
# Step 3. Initialize the embeddings model
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_mac')
embeddings = OpenAIEmbeddings(model='text-embedding-3-large', api_key=OPENAI_API_KEY)

# Step 4. Index the document chunks in Chroma vector store
db = Chroma.from_documents(documents=documents, embedding=embeddings)
print("Documents indexed in Chroma successfully.")

Documents indexed in Chroma successfully.


In [10]:
# Step 5. Define a retriever for similarity search
retriever = db.as_retriever(search_type='similarity', search_kwargs={'k':3})    # Retrieve top 3 relevant chunks

In [16]:
# Step 6: Define the prompt template for the LLM
prompt = ChatPromptTemplate.from_template(
    """
    You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise.
    Question: {question}
    Context: {context}
    Answer:
    """  
)

# Step 7. Initialize the ChatOpenAI model

# Set up obeservability
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = keyring.get_password('langsmith', 'learning_agent')
os.environ['LANGCHAIN_PROJECT'] = "pr-stupendous-hood-8"
model = ChatOpenAI(model='gpt-4o-mini', temperature=0, api_key=OPENAI_API_KEY)

In [17]:
# Step 8: Set up the Retrieval-Augmented Generation(RAG) chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [20]:
# Step 9. Ask a question and generate a response
question = "Can you explain what FAISS is used for?"
for chunk in rag_chain.stream(question):
    print(chunk, end="", flush=True)

FAISS (Facebook AI Similarity Search) is a library designed for efficient vector similarity search, particularly in large collections of embedding vectors. It provides a variety of indexing methods and tools for searching, clustering, compressing, and transforming vectors, making it suitable for applications like natural language processing and data mining. FAISS is widely used in both industry and research for tasks that require fast and scalable nearest neighbor searches.